# End to End Pure Streaming Data-Pipeline for Apartment Table Using Spark Structured Streaming on Databricks

###### Description: In this notebook we read apartment state rows from incoming csv files into a streamig dataframe, transform (clean, cast, rename) the data, add/update the latest state to a Databricks Delta table
###### Objective: (incoming csv files) --> "apartment_streamingDF" --> "results_df" --> "apartment_data"

In [3]:
import requests
import json
import optimus as op
import phonenumbers 
import re
import datetime
import time

from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql import HiveContext
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W
from pyspark.sql import DataFrame
from pyspark.sql.functions import lit
from pyspark.sql.functions import rank, col

In [4]:
# Schema for Apartment JSON
apartment_schema = StructType([
            StructField("Building_id", IntegerType(), True),
            StructField("Apartment_number", IntegerType(), True),
            StructField("Type", StringType(), True),
            StructField("Rent_fee", StringType(), True),
            StructField("Building_name", StringType(), True),
            StructField("Appt_details", StringType(), True),
            StructField("event_time", TimestampType(), True),
            StructField("fetch_time", StringType(), True)])

apartment_udf_schema = StructType([
            StructField("Apartment_number", IntegerType(), True),
            StructField("Type", StringType(), True),
            StructField("Rent_fee", StringType(), True),
            StructField("Building_name", StringType(), True),
            StructField("Appt_details", StringType(), True),
            StructField("event_time", TimestampType(), True)])

###### Description: Get apartment csv files as a streaming "apartment_streamingDF" and process it on the fly and get transformed stream "apartment_df"
###### Objective: (incoming csv files) --> "apartment_streamingDF" --> "apartment_df"

In [6]:
# Get Apartment Steaming DataFrame from csv files

# streaming starts here by reading the input files 
apartment_Path = "/FileStore/apartment/apartment/inprogress/"
apartment_streamingDF = (
  spark
    .readStream
    .schema(apartment_schema)
    .option("maxFilesPerTrigger", "1")
    .option("header", "true")
    .option("multiLine", "true")
    .csv(apartment_Path)
)
# Clear invalid rows
apartment_df = apartment_streamingDF.select("*").where("Building_id IS NOT NULL")
# Instantiation of DataTransformer class:
transformer = op.DataFrameTransformer(apartment_df)
# Replace NA with 0's
transformer.replace_na(0.0, columns="*")
# Clear accents: clear_accents only from name column and not everywhere 
transformer.clear_accents(columns='*')
# Remove special characters:  From all Columns 
transformer.remove_special_chars(columns=['Type', 'Rent_fee', 'Building_name', 'Appt_details'])

##### This function parses the corresponding columns into a single column

In [8]:
def my_fun(Apartment_number, Type, Rent_fee, Building_name, Appt_details, event_time):
  return zip(Apartment_number, Type, Rent_fee, Building_name, Appt_details, event_time)

udf_Fun = udf(my_fun, ArrayType(apartment_udf_schema))

In [9]:
intermediate_df = ( apartment_df.withWatermark("event_time", "10 seconds")
            .groupBy("Building_id")
            .agg(F.collect_list("Apartment_number").alias("Apartment_number"),
                 F.collect_list("Type").alias("Type"),
                 F.collect_list("Rent_fee").alias("Rent_fee"),
                 F.collect_list("Building_name").alias("Building_name"),
                 F.collect_list("Appt_details").alias("Appt_details"),
                 F.collect_list("event_time").alias("event_time"), 
                 F.max("event_time").alias("latest_event_time"))
            .select("Building_id", 
                    F.explode(udf_Fun(F.column("Apartment_number"), 
                                      F.column("Type"), 
                                      F.column("Rent_fee"), 
                                      F.column("Building_name"), 
                                      F.column("Appt_details"), 
                                      F.column("event_time")))
                    .alias("data"), "latest_event_time"))

##### Filter the data where event_time is latest

In [11]:
results_df = (intermediate_df
              .select("Building_id", 
                      "data.Apartment_number", 
                      "data.Type", 
                      "data.Rent_fee", 
                      "data.Building_name", 
                      "data.Appt_details", 
                      "data.event_time", 
                      "latest_event_time")
              .where("data.event_time=latest_event_time")).orderBy("Building_id")

##### Display final result
###### This result shows the latest state of all the unique Building_id

In [13]:
display(results_df)

##### Below cells are optional if external functionality or storage is needed

###### Write the stream to a Databricks Delta table for storage

In [16]:
streaming_query = (results_df.writeStream
 .format("delta")
 .outputMode("complete")
 .option("mergeSchema", "true")
 .option("checkpointLocation", "/delta/apartment/_checkpoints/streaming-agg")
 .start("/delta/apartment/apartment_data"))

#### Read the Delta Table as a Static or Streaming DataFrame
#### This dataframe wil always be Up-To-Date

In [18]:
apartment_data = spark.read.format("delta").load("/delta/apartment/apartment_data").orderBy("Building_id")

In [19]:
display(apartment_data)

### Do Some Live Streaming Graphs

In [21]:
apartment_data_stream = spark.readStream.format("delta").load("/delta/apartment/apartment_data")

In [22]:
display(apartment_data_stream.groupBy("Type").count())